In [27]:
import numpy as np
import pandas as pd
import requests
import re

In [13]:
def get_data(url, number):
    headers = {'User-agent': 'eamonious'}
    end_addedum = '&sort=des&size=1000'
    base_value = 1545243580
    base_url = url + str(base_value) + end_addedum
    res = requests.get(base_url, headers=headers)
    json = res.json()
    comments = pd.DataFrame(json['data'])
    comments = comments[['body','created_utc','id','parent_id','score','subreddit']]
    comments['parent_id'] = comments['parent_id'].map(lambda x: x if 't3_' in x else 0)
    comments = comments[comments['parent_id']!=0]
    comments = comments[comments['body']!='[removed]']
    for i in range(1, number):
        headers = {'User-agent': 'eamonious'}
        temp_url = url + str(base_value - i*1036800) + end_addedum
        print(temp_url)
        print("https://api.pushshift.io/reddit/search/comment/?subreddit=askmen&before={}&sort=des&size=1000".format(1545243580 - i*1036800))
        res = requests.get(temp_url, headers=headers)
        json = res.json()
        commentbloc = pd.DataFrame(json['data'])
        commentbloc = commentbloc[['body','created_utc','id','parent_id','score','subreddit']]
        commentbloc['parent_id'] = commentbloc['parent_id'].map(lambda x: x if 't3_' in x else 0)
        commentbloc = commentbloc[commentbloc['parent_id']!=0]
        commentbloc = commentbloc[commentbloc['body']!='[removed]']
        comments = pd.concat([comments, commentbloc], ignore_index=True)
    
    return comments

In [14]:
MEN_URL = "https://api.pushshift.io/reddit/search/comment/?subreddit=askmen&before="
WOMEN_URL = "https://api.pushshift.io/reddit/search/comment/?subreddit=askwomen&before="

In [16]:
men_comments = get_data(MEN_URL, 80)

In [17]:
men_comments

,body,created_utc,id,parent_id,score,subreddit
0,I was 23 but I went with my ~29 year old cowor...,1545243578,ec4lbi4,t3_a7oy9v,5,AskMen
1,"Portland, OR.\n\nThe city itself is now unaffo...",1545243546,ec4la0n,t3_a7mkui,3,AskMen
2,"nope. ""the cats goodbye"" watch how a c...",1545243536,ec4l9lm,t3_a7fe60,1,AskMen
3,Drunk as fuck me during an unintended one nigh...,1545243524,ec4l90i,t3_a79zu9,3,AskMen
4,There was this one time when I went over one o...,1545243449,ec4l5g6,t3_a7kmvc,134,AskMen
...,...,...,...,...,...,...
8531,"You really, really need to get this book: [Ful...",1463329753,d36fx8l,t3_4jeo8t,2,AskMen
8532,[deleted],1463329697,d36fw2r,t3_4jgp5r,1,AskMen
8533,"Probably like once a day, I make a few weird f...",1463329690,d36fvwv,t3_4jgp5r,1,AskMen
8534,/r/blackpeopletwitter,1463329670,d36fvh7,t3_4jgc80,0,AskMen


In [18]:
women_comments = get_data(WOMEN_URL, 80)

In [21]:
def further_processing(df, subreddit):

    df.dropna(inplace=True)
    df.drop_duplicates('id', inplace=True)

    df = df[df['body']!='[deleted]']
    df = df[df['body']!='\\[removed\]']
    df['body'] = df['body'].map(lambda x: 0 if 'has been removed' in x else x)
    df['body'] = df['body'].map(lambda x: 0 if subreddit in str(x) else x)
    df['body'] = df['body'].map(lambda x: 0 if 'emoved' in str(x)[0:10] else x)
    
    df = df[df['body']!=0]

    df = df[df['body']!='Please feel free to respond based on the genders that you find attractive. This question is not limited to women who date men.']

    return df

In [22]:
processed_men_comments = further_processing(men_comments, 'AskMen')
processed_women_comments = further_processing(women_comments, 'AskWomen')

In [23]:
processed_men_comments.shape, processed_women_comments.shape

((7452, 6), (7549, 6))

In [24]:
comments = pd.concat([processed_men_comments, processed_women_comments])
comments = comments.reset_index(drop=True)
comments.head()

,body,created_utc,id,parent_id,score,subreddit
0,I was 23 but I went with my ~29 year old cowor...,1545243578,ec4lbi4,t3_a7oy9v,5,AskMen
1,"Portland, OR.\n\nThe city itself is now unaffo...",1545243546,ec4la0n,t3_a7mkui,3,AskMen
2,"nope. ""the cats goodbye"" watch how a c...",1545243536,ec4l9lm,t3_a7fe60,1,AskMen
3,Drunk as fuck me during an unintended one nigh...,1545243524,ec4l90i,t3_a79zu9,3,AskMen
4,There was this one time when I went over one o...,1545243449,ec4l5g6,t3_a7kmvc,134,AskMen


In [28]:
#This function selects any consecutive combination of \r's and \n's in a bloc of text, 
#and replaces that selection with a single space.
def replace_linebreaks_w_space(x):
	return re.sub('([\r\n]+)',' ',x)

#This function selects any stretch of two or more consecutive spaces in a bloc of text,
#and replaces that selection with a single space.
def replace_multispace_w_space(x):
	return re.sub('([ ]{2,})',' ',x)

#Here we take every comment and apply the two functions to it.
comments['body'] = comments['body'].map(replace_linebreaks_w_space)
comments['body'] = comments['body'].map(replace_multispace_w_space)

In [29]:
#Strip away any spaces at the beginning or end of each comment, splits the comment into a list of words, 
#and returns the length of that list (i.e.; the number of words in the comment)
comments['word_length'] = comments['body'].map(lambda x: len(x.strip().split(' ')))

In [30]:
comments.head()

,body,created_utc,id,parent_id,score,subreddit,word_length
0,I was 23 but I went with my ~29 year old cowor...,1545243578,ec4lbi4,t3_a7oy9v,5,AskMen,23
1,"Portland, OR. The city itself is now unafforda...",1545243546,ec4la0n,t3_a7mkui,3,AskMen,50
2,"nope. ""the cats goodbye"" watch how a cat says ...",1545243536,ec4l9lm,t3_a7fe60,1,AskMen,28
3,Drunk as fuck me during an unintended one nigh...,1545243524,ec4l90i,t3_a79zu9,3,AskMen,16
4,There was this one time when I went over one o...,1545243449,ec4l5g6,t3_a7kmvc,134,AskMen,192


In [31]:
comments = comments[comments['word_length']>=4]
len(comments)

13963

In [32]:
comments['subreddit'].value_counts()

AskMen      13848
AskWomen      115
Name: subreddit, dtype: int64

In [33]:
comments.to_csv('comments_final.csv',index=False)